In [56]:
import json

In [81]:
def corpus_divider(file_names):
    statistics = dict()
    for file in file_names:
        ner_annotations = list()
        with open("vallakohtufailid_json/" + file, 'r', encoding="UTF-8)") as f:
            data = json.load(f)
            for dictionary in data.get('layers')[0].get('spans'):
                ner_annotations.append(dictionary.get('annotations')[0].get('nertag'))

        statistics_for_file = dict()
        for annotation in set(ner_annotations):
            statistics_for_file[str(annotation)] = ner_annotations.count(annotation)
        statistics[file] = statistics_for_file

    return statistics

def calculate_proportions(file_names):
    statistics = corpus_divider(file_names)
    all_annotations = list()
    for item in statistics.values():
        for key in item:
            appendable = key
            for i in range(0, item[key] + 1):
                all_annotations.append(appendable)
    
    proportions = dict()
    for file in statistics:
        for key in statistics[file].keys():
            proportion = all_annotations.count(key) / len(all_annotations)
            proportions[key] = round(proportion, 2)
        
    return proportions
    
#TODO: def calculate_score(file_names): 
#TODO: def generate_random_division(file_names):

names = ['Harju_Hageri_Kohila_id2805_1882a.json', 'Harju_Rapla_Rapla_id18416_1868a.json',\
        'L22ne_Martna_Martna_id17524_1871a.json', 'P2rnu_Audru_V6lla_id5904_1878a.json']

print(calculate_proportions(names))

{'PER': 0.78, 'LOC_ORG': 0.11, 'ORG': 0.05, 'LOC': 0.03, 'MISC': 0.02}
